In [ ]:
#Thanks to: https://www.curiousily.com/posts/object-detection-on-custom-dataset-with-tensorflow-2-and-keras-using-python/

In [ ]:
#MLFlow tutorial https://databricks.com/blog/2018/09/21/how-to-use-mlflow-to-reproduce-results-and-retrain-saved-keras-ml-models.html

In [1]:
from mlflow import log_metric, log_param, log_artifact

In [2]:
!pip3 install sklearn

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [3]:
import pandas as pd
import urllib.request
from PIL import Image
from sklearn.model_selection import train_test_split

In [4]:
!pip3 install gdown

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [5]:
!gdown --id 1mTtB8GTWs74Yeqm0KMExGJZh1eDbzUlT --output indian_number_plates.json

Downloading...
From: https://drive.google.com/uc?id=1mTtB8GTWs74Yeqm0KMExGJZh1eDbzUlT
To: C:\Users\lafacero\Documents\Jupyter Notebooks\indian_number_plates.json

  0%|          | 0.00/92.5k [00:00<?, ?B/s]
100%|##########| 92.5k/92.5k [00:00<00:00, 93.0MB/s]


In [6]:
plates_df = pd.read_json('indian_number_plates.json', lines=True)

c:\users\lafacero\appdata\local\programs\python\python37\lib\site-packages\botocore\vendored\requests\packages\urllib3\_collections.py:1: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping, MutableMapping


In [7]:
import os
os.makedirs("number_plates", exist_ok=True)

In [8]:
dataset = dict()
dataset["image_name"] = list()
dataset["top_x"] = list()
dataset["top_y"] = list()
dataset["bottom_x"] = list()
dataset["bottom_y"] = list()
dataset["class_name"] = list()

In [9]:
#download train and convert the data into the format that Keras Retina understands
counter = 0
for index, row in plates_df.iterrows():
    img = urllib.request.urlopen(row["content"])
    img = Image.open(img)
    img = img.convert('RGB')
    img.save(f'number_plates/licensed_car_{counter}.jpeg', "JPEG")

    dataset["image_name"].append(
      f'number_plates/licensed_car_{counter}.jpeg'
    )

    data = row["annotation"]

    width = data[0]["imageWidth"]
    height = data[0]["imageHeight"]

    dataset["top_x"].append(int(round(data[0]["points"][0]["x"] * width)))
    dataset["top_y"].append(int(round(data[0]["points"][0]["y"] * height)))
    dataset["bottom_x"].append(int(round(data[0]["points"][1]["x"] * width)))
    dataset["bottom_y"].append(int(round(data[0]["points"][1]["y"] * height)))
    
    dataset["class_name"].append("license_plate")

    counter += 1

In [10]:
print("Downloaded {} car images.".format(counter))


Downloaded 237 car images.


In [11]:
df = pd.DataFrame(dataset)

In [12]:
df

,image_name,top_x,top_y,bottom_x,bottom_y,class_name
0,number_plates/licensed_car_0.jpeg,582,274,700,321,license_plate
1,number_plates/licensed_car_1.jpeg,80,382,288,449,license_plate
2,number_plates/licensed_car_2.jpeg,41,219,115,260,license_plate
3,number_plates/licensed_car_3.jpeg,217,313,359,359,license_plate
4,number_plates/licensed_car_4.jpeg,209,220,866,554,license_plate
5,number_plates/licensed_car_5.jpeg,106,280,261,335,license_plate
6,number_plates/licensed_car_6.jpeg,227,241,444,315,license_plate
7,number_plates/licensed_car_7.jpeg,267,248,396,320,license_plate
8,number_plates/licensed_car_8.jpeg,644,508,893,578,license_plate
9,number_plates/licensed_car_9.jpeg,674,365,958,572,license_plate


In [13]:
RANDOM_SEED = 42
train_df, test_df = train_test_split(df,test_size=0.2,random_state=RANDOM_SEED)

In [ ]:
train_df

In [14]:
ANNOTATIONS_FILE = 'annotations.csv'
CLASSES_FILE = 'classes.csv'
#Pandas to write the annotations file, excluding the index and header:
train_df.to_csv(ANNOTATIONS_FILE, index=False, header=None)

In [15]:
#regular old file writer for the classes:

classes = set(['license_plate'])

with open(CLASSES_FILE, 'w') as f:
  for i, line in enumerate(sorted(classes)):
    f.write('{},{}\n'.format(line,i))

In [16]:
os.makedirs("snapshots", exist_ok=True)

In [ ]:
#downloading generic pre-trained-model trained on COCO dataset
!gdown --id 1wPgOBoSks6bTIs9RzNvZf6HWROkciS8R --output snapshots\resnet50_csv_10.h5

In [ ]:
!git clone https://github.com/fizyr/keras-retinanet.git

In [18]:
#Note: install VS 14 setup tools https://visualstudio.microsoft.com/downloads/
import pandas as pd
import urllib.request
from keras_retinanet import models

ModuleNotFoundError: No module named 'keras_retinanet'

In [17]:
#RESUME FROM HERE!
PRETRAINED_MODEL = './snapshots/_pretrained_model.h5'
#training model
!keras_retinanet/bin/train.py \
 --freeze-backbone \
 --random-transform \
 --weights {PRETRAINED_MODEL} \
 --batch-size 8 \
 --steps 500 \
 --epochs 10 \
 csv annotations.csv classes.csv

'keras_retinanet' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
#loading the model 

model_path = os.path.join('snapshots', sorted(os.listdir('snapshots'), reverse=True)[0])

model = models.load_model(model_path, backbone_name='resnet50')
model = models.convert_model(model)

In [ ]:
#convert the classes into a format that will be useful later:
labels_to_names = pd.read_csv(
  CLASSES_FILE,
  header=None
).T.loc[0].to_dict()

In [ ]:
#using model:
def predict(image):
  image = preprocess_image(image.copy())
  image, scale = resize_image(image)

  boxes, scores, labels = model.predict_on_batch(
    np.expand_dims(image, axis=0)
  )

  boxes /= scale

  return boxes, scores, labels